In [12]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [49]:
spark = SparkSession.builder \
    .appName("UberTripsKafkaConsumer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.sql.session.timeZone", "UTC") \
    .config("spark.sql.streaming.checkpointLocation.maxFileSize", "1000000") \
    .config("spark.sql.streaming.schemaInference", "false") \
    .getOrCreate()

In [61]:
df = spark.read.parquet("D:/Just Data/Uber Real-Time Analytics Pipeline/cleaned_uber_trips")
uncleaned_df = spark.read.parquet("D:/Just Data/Uber Real-Time Analytics Pipeline/uber_trips")
raw_df = spark.read.parquet("D:/Just Data/Uber Real-Time Analytics Pipeline/yellow_tripdata_2025-01.parquet")

In [62]:
df.select([count(col(c)).alias(c) for c in df.columns]).show()

+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+---------------+------------+--------------+---------+----------+---------------------+------------+-------------+---------------+------------+
|trip_id|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|Airport_fee|cbd_congestion_fee|kafka_timestamp|kafka_offset|ingestion_time|pickup_ts|dropoff_ts|trip_duration_minutes|total_amount|avg_speed_mph|is_high_quality|processed_at|
+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+--------

In [ ]:
uncleaned_df.select([count(col(c)).alias(c) for c in uncleaned_df.columns]).show()

In [63]:
df.orderBy('trip_id').show()

+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+--------------------+------------+--------------------+-------------------+-------------------+---------------------+------------+------------------+---------------+--------------------+
|trip_id|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|Airport_fee|cbd_congestion_fee|     kafka_timestamp|kafka_offset|      ingestion_time|          pickup_ts|         dropoff_ts|trip_duration_minutes|total_amount|     avg_speed_mph|is_high_quality|        processed_at|
+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+-

In [57]:
uncleaned_df.orderBy('trip_id').show()

+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------------+------------+-------------------+-------------------+--------------------+
|trip_id|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|     kafka_timestamp|kafka_offset|          pickup_ts|         dropoff_ts|      ingestion_time|
+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+--------------------+------------+------------

In [53]:
raw_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       1| 2025-01-01 00:18:38|  2025-01-01 00:26:59|              1|          1.6|         1|                 N|         229|    

In [88]:
df.agg(max('trip_id').alias('mini')).show()

+----+
|mini|
+----+
| 703|
+----+



In [34]:
raw_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



In [66]:
df.groupBy('trip_id').agg(count('*').alias('counter')).filter(col('counter') > 1).show()

+-------+-------+
|trip_id|counter|
+-------+-------+
+-------+-------+



In [87]:
df.filter(col('trip_id') == 450).show()

+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+--------------------+-----------+------------------+--------------------+------------+--------------------+-------------------+-------------------+---------------------+------------+-----------------+---------------+--------------------+
|trip_id|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|Airport_fee|cbd_congestion_fee|     kafka_timestamp|kafka_offset|      ingestion_time|          pickup_ts|         dropoff_ts|trip_duration_minutes|total_amount|    avg_speed_mph|is_high_quality|        processed_at|
+-------+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+---